# 2024 NASACT Training xlwings demo
By: Ohio Auditor of State (Jesse Carroll/Joel Thompson)

## Import Python Libraries

In [1]:
import pandas as pd  #what does 'as' do?
import xlwings as xw  #what is xlwings?
from pathlib import Path
from shutil import copyfile

## Import the data we want to enter into our template

In [3]:
# store path in a variable called current file
current_file = r"random_GL_Reports_2023.xlsx"

# import the GL_Transactions tab from the workbook
df = pd.read_excel(current_file , sheet_name='GL_Transactions')


first date is 2022-07-01 00:00:00
last date is 2023-06-30 00:00:00


In [ ]:

#store the current date in a variable called "today"
today = pd.to_datetime('today')

#sort Tran_Date in ascending order
df = df.sort_values(by='TranDate', ascending=True)

# store the first TranDate in a variable called "first_date"
first_date = df['TranDate'].iloc[0]

#print "first date is" and the value of first_date
print(f'first date is {first_date}')

#store the last TranDate in a variable called "last_date"
last_date = df['TranDate'].iloc[-1]

#print "last date is" and the value of last_date
print(f'last date is {last_date}')

#total the ExpAmt columns and store in total_exp
total_exp = df['ExpAmt'].sum()

#total the RevAmt columns and store in total_rev
total_rev = df['RevAmt'].sum()


entering the variables into the excel template

In [4]:
destination = rf'D:\Data\DITA\Training\Results\results.xlsx'
current_template = rf'D:\Data\DITA\Training\template.xlsx'

#copy the current_template to the destination
copyfile(current_template, destination)

with xw.App(visible=False) as xl:
        
        wb = xw.Book(destination)
        xl = xw.apps.active.api

        #wb.sheets.add('RecordedData')
        template_tab = wb.sheets('tab1')
        wb.sheets['tab1'].api.Visible = True
        #tameplate_tab cell B3 = current_file
        template_tab.range('B3').value = current_file
        #template_tab cell B5 = today
        template_tab.range('B5').value = today
        #set cell B5 to date format
        template_tab.range('B5').number_format = "mm/dd/yyyy"
        #template_tab cell B7 = first_date
        template_tab.range('B7').value = first_date
        #set cell B7 to date format
        template_tab.range('B7').number_format = "mm/dd/yyyy"
        #template_tab cell B9 = last_date
        template_tab.range('B9').value = last_date
        #set cell B9 to date format
        template_tab.range('B9').number_format = "mm/dd/yyyy"
        #template_tab cell B11 = total_exp
        template_tab.range('B11').value = total_exp
        #set sell B11 to currency format with 2 decimals
        template_tab.range('B11').number_format = "$#,##0.00"
        #template_tab cell B13 = total_rev
        template_tab.range('B13').value = total_rev
        #set cell B13 to currency format
        template_tab.range('B13').number_format = "$#,##0.00"

        gl_trans_tab = wb.sheets('GL_Transactions')
        #paste df into cell A1 in gl_trans_tab
        gl_trans_tab.range('A1').options(index=False).value = df

        #MORE FORMATTING FUN - remove the comments to see the magic

        #in the template_tab, autofit columns B and H and K
        template_tab.range('B:B').autofit()
        template_tab.range('H:H').autofit()
        template_tab.range('K:K').autofit()

        gl_trans_tab.used_range.api.AutoFilter(Field:=1)

        #bold the text in the first row
        gl_trans_tab.range('1:1').api.Font.Bold = True

        #autofit the columns in gl_trans_tab
        gl_trans_tab.range('A:AD').autofit()

        wb.save(destination)
        wb.close()